# Introduction

There are two tasks: an algorithm design and a data analysis task. Both tasks were recently encoutered tasks by the data science team at SST. Above all, we are looking for clean, readable code and adherence to standard data science practices. One resource that may help you with the former is a Raymond Hettinger talk on the same subject (https://www.youtube.com/watch?v=OSGv2VnC0go). You do not have to watch it for these exercises, although the principles outlined will be beneficial to any Python developer.

All constraints are explicitly stated, so if something is ambiguous, then you are free to answer as you see fit. Our intent is for this to take no longer than 60-90 minutes, so if you find yourself taking longer, then please answer what you can and explain what you found challenging.


## 1. Algorithm


##### Background
We are looking to visualize a collection of continuous events on a timeline. Some of these events may be overlapping (e.g. end time of event 1 could fall between the start time and end time of event 2). As such, we want to stack these events on different levels, such that there is no overlap in the visualization. 

For example, if three events have start and end time pairs (0, 50), (20, 60), and (55, 100), then the solution could be visualized as the following:

```
                          20                    60
                          |                     |
level 1                   ----------------------
level 0            --------------------    -----------------
                   |                  |    |               |
                   0                  50  55              100  
```
##### Constraints
- You must use the minimum number of required levels to stack the events such that they are not overlapping
- An event must be assigned to the **lowest** unoccupied level
- If two events have the same time, the event in the lowest index in the input list must be placed in the lowest level

##### Input
A **list** of **dictionaries**, where each dictionary has two keys: *'start_time'* and *'end_time'*. The corresponding values for *'start_time'* and *'end_time'* are non-negative integers. 

Sample:
```
sample_input = [
    {
        'start_time': 0,
        'end_time': 50
    }, 
    {
        'start_time': 20,
        'end_time': 60
    }, 
    {
        'start_time': 55,
        'end_time': 100
    }
]
```

##### Output
The output must have the same format as the input (list of dictionaries), but with an additional key:value pair in each dictionary. The key will be named *'level'* and the value will be the integer level assigned to that event.

Sample:
```
sample_input = [
    {
        'start_time': 0,
        'end_time': 50,
        'level': 0
    }, 
    {
        'start_time': 20,
        'end_time': 60,        # event 0 occupies level 0 until time = 50,
        'level': 1             # so level 1 is the lowest unoccupied level
    }, 
    {
        'start_time': 55,
        'end_time': 100,       # event 0 concluded at time = 50, so
        'level': 0             # level 0 is now available again
    }
]
```

##### Given
- end time >= start time
- input list is sorted by start time


In [1]:
import numpy as np
from collections import OrderedDict


sample_input = [
    {
        'start_time': 0,
        'end_time': 50
    }, 
    {
        'start_time': 20,
        'end_time': 60
    }, 
    {
        'start_time': 55,
        'end_time': 100
    },
    {
        'start_time': 35,
        'end_time': 100
    }
]


def add_levels(events):
    # this dictionary is to keep track of the maximum end time of each level
    end_time_by_level = OrderedDict()
    max_level = 0
    for event in events:
        current_level = 0
        if events.index(event) == 0:
            event['level'] = current_level
            end_time_by_level[event['level']] = event['end_time']
        else:
            for i in range(len(end_time_by_level)):
                # if start time of the current event is greater than the maximum of end time of previous events
                if event['start_time'] > end_time_by_level[i]:
                    # assign the level and change the maximum value of that level
                    event['level'] = list(end_time_by_level.keys())[i]
                    end_time_by_level[event['level']] = event['end_time']
                    break
                else:
                    # if the event is on the max level, increase the max level by one and move to the next level
                    if max_level == current_level:
                        max_level += 1
                        current_level += 1
                        event['level'] = current_level
                        end_time_by_level[event['level']] = event['end_time']
                    # if the event is not on the max level, move to the next level
                    else:
                        current_level += 1
                        event['level'] = current_level
    return events


print(add_levels(sample_input))

[{'start_time': 0, 'end_time': 50, 'level': 0}, {'start_time': 20, 'end_time': 60, 'level': 1}, {'start_time': 55, 'end_time': 100, 'level': 0}, {'start_time': 35, 'end_time': 100, 'level': 2}]


It took me longer than intended, because I found tracking multiple floors in the right order challenging. I could visualize the algoritmn and procedures in my head, but the actual coding and debugging took slightly longer than I hoped.

## 2. EDA

##### Background
In the attached CSV, `tasks.csv`, there are a collection of tasks performed by different analysts. 

Different task types are denoted by different stage IDs, and different analysts are denoted by different analyst IDs. Not all analysts perform the same task types (i.e., stage IDs).

The nature of the task type is not important, but note that the file associated with a particular task has a duration (`file_duration` column).

##### Output
Who is/are the most productive analyst(s)? Please write your answer at the end of your research, and include 1-2 sentences with your reasoning.

In [2]:
import pandas as pd
import datetime as dt


tasks = pd.read_csv('tasks.csv')
tasks.head()

,task_id,file_duration,stage_id,analyst_id,assigned_time,completion_time
0,28285,15340,27,19,2020-08-06 11:22:09,2020-08-06 11:59:29
1,28145,14277,107,19,2020-08-06 12:44:05,2020-08-06 16:01:04
2,27915,18381,57,99,2020-08-06 13:42:08,2020-08-10 12:30:39
3,45255,86400,17,139,2020-08-12 09:48:15,2020-08-12 10:11:27
4,28445,28433,107,149,2020-08-06 12:36:37,2020-08-07 09:23:20


In [3]:
tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   task_id          412 non-null    int64 
 1   file_duration    412 non-null    int64 
 2   stage_id         412 non-null    int64 
 3   analyst_id       412 non-null    int64 
 4   assigned_time    412 non-null    object
 5   completion_time  412 non-null    object
dtypes: int64(4), object(2)
memory usage: 19.4+ KB


In [4]:
tasks['duration'] = (pd.to_datetime(tasks['completion_time']) - pd.to_datetime(tasks['assigned_time']))
tasks.sort_values(by='duration', inplace=True)

In [5]:
tasks.head(15)

,task_id,file_duration,stage_id,analyst_id,assigned_time,completion_time,duration
401,49915,86400,17,159,2020-08-28 10:48:53,2020-08-28 10:48:54,0 days 00:00:01
397,49895,86400,17,159,2020-08-28 10:47:18,2020-08-28 10:47:19,0 days 00:00:01
410,49905,86400,17,159,2020-08-28 10:44:02,2020-08-28 10:44:03,0 days 00:00:01
372,48445,18149,107,69,2020-08-25 14:40:22,2020-08-25 14:41:13,0 days 00:00:51
343,48455,23917,107,69,2020-08-25 14:41:14,2020-08-25 14:42:34,0 days 00:01:20
39,34145,16109,27,69,2020-08-07 09:26:43,2020-08-07 09:28:15,0 days 00:01:32
322,48555,86400,17,149,2020-08-24 09:31:23,2020-08-24 09:33:40,0 days 00:02:17
332,48565,86400,17,149,2020-08-24 09:38:15,2020-08-24 09:41:19,0 days 00:03:04
210,46885,86400,17,149,2020-08-17 10:48:34,2020-08-17 10:52:11,0 days 00:03:37
126,44615,86400,17,149,2020-08-10 10:37:37,2020-08-10 10:41:15,0 days 00:03:38


The first three lines look really unrealistic; he/she finished 3 tasks within 3 seconds. I'm dropping those rows.

In [6]:
tasks.tail(15)

,task_id,file_duration,stage_id,analyst_id,assigned_time,completion_time,duration
227,45755,24625,57,69,2020-08-14 12:37:56,2020-08-17 16:07:05,3 days 03:29:09
38,44145,28433,57,29,2020-08-07 09:59:25,2020-08-10 14:22:23,3 days 04:22:58
336,47715,31947,37,119,2020-08-21 08:51:27,2020-08-24 14:28:33,3 days 05:37:06
272,47735,18258,37,49,2020-08-20 16:29:49,2020-08-24 12:51:01,3 days 20:21:12
304,45235,29854,37,109,2020-08-20 11:38:05,2020-08-24 09:49:55,3 days 22:11:50
279,47625,15648,57,19,2020-08-20 11:07:15,2020-08-24 09:25:53,3 days 22:18:38
281,47705,31947,57,59,2020-08-20 13:23:26,2020-08-24 11:50:46,3 days 22:27:20
2,27915,18381,57,99,2020-08-06 13:42:08,2020-08-10 12:30:39,3 days 22:48:31
79,36355,18381,37,109,2020-08-07 12:49:30,2020-08-11 12:20:16,3 days 23:30:46
84,45115,17677,57,29,2020-08-13 11:07:44,2020-08-17 14:21:00,4 days 03:13:16


In [7]:
tasks.drop([401, 397, 410], inplace=True)
tasks.reset_index(drop=True, inplace=True)
tasks.head()

,task_id,file_duration,stage_id,analyst_id,assigned_time,completion_time,duration
0,48445,18149,107,69,2020-08-25 14:40:22,2020-08-25 14:41:13,0 days 00:00:51
1,48455,23917,107,69,2020-08-25 14:41:14,2020-08-25 14:42:34,0 days 00:01:20
2,34145,16109,27,69,2020-08-07 09:26:43,2020-08-07 09:28:15,0 days 00:01:32
3,48555,86400,17,149,2020-08-24 09:31:23,2020-08-24 09:33:40,0 days 00:02:17
4,48565,86400,17,149,2020-08-24 09:38:15,2020-08-24 09:41:19,0 days 00:03:04


In [8]:
by_file_duration = tasks[['file_duration', 'analyst_id']].groupby(by='analyst_id', as_index=False).sum()
by_tasks_count = tasks[['file_duration', 'analyst_id']].groupby(by='analyst_id', as_index=False).count()
by_duration = tasks[['duration', 'analyst_id']].groupby(by='analyst_id', as_index=False).sum()

In [9]:
overall_performance = by_file_duration.merge(by_tasks_count, on='analyst_id').merge(by_duration, on='analyst_id')
overall_performance.rename(columns={'file_duration_x':'file_duration', 'file_duration_y':'tasks_completed'}, inplace=True)

overall_performance['average file_duration'] = overall_performance['file_duration']/overall_performance['tasks_completed']
overall_performance['average completion time'] = overall_performance['duration']/overall_performance['tasks_completed']

overall_performance.sort_values('average file_duration', inplace=True)
overall_performance

,analyst_id,file_duration,tasks_completed,duration,average file_duration,average completion time
2,39,309758,23,20 days 10:43:45,13467.739130,0 days 21:20:09.782608695
4,59,557664,33,20 days 22:31:04,16898.909091,0 days 15:13:40.121212121
0,19,287842,17,19 days 21:02:52,16931.882353,1 days 04:03:41.882352941
1,29,224654,13,13 days 05:35:00,17281.076923,1 days 00:25:46.153846153
5,69,345625,20,17 days 08:20:16,17281.250000,0 days 20:49:00.800000
6,99,139050,8,13 days 13:54:20,17381.250000,1 days 16:44:17.500000
3,49,214509,12,20 days 00:56:44,17875.750000,1 days 16:04:43.666666666
8,119,507488,26,20 days 19:46:16,19518.769231,0 days 19:13:19.076923076
7,109,354591,15,21 days 03:51:08,23639.400000,1 days 09:51:24.533333333
9,139,4316976,108,17 days 00:38:26,39972.000000,0 days 03:47:01.351851851


In [10]:
overall_performance.sort_values('average completion time', ascending=True)

,analyst_id,file_duration,tasks_completed,duration,average file_duration,average completion time
13,319,259200,3,0 days 01:41:59,86400.000000,0 days 00:33:59.666666666
12,309,172800,2,0 days 02:00:13,86400.000000,0 days 01:00:06.500000
10,149,6251452,125,16 days 19:22:29,50011.616000,0 days 03:13:37.192000
9,139,4316976,108,17 days 00:38:26,39972.000000,0 days 03:47:01.351851851
11,299,209382,4,0 days 20:30:03,52345.500000,0 days 05:07:30.750000
4,59,557664,33,20 days 22:31:04,16898.909091,0 days 15:13:40.121212121
8,119,507488,26,20 days 19:46:16,19518.769231,0 days 19:13:19.076923076
5,69,345625,20,17 days 08:20:16,17281.250000,0 days 20:49:00.800000
2,39,309758,23,20 days 10:43:45,13467.739130,0 days 21:20:09.782608695
1,29,224654,13,13 days 05:35:00,17281.076923,1 days 00:25:46.153846153


In [25]:
tasks.loc[tasks['analyst_id'] == 149][['file_duration', 'duration']].describe()

,file_duration,duration
count,125.0000,125
mean,50011.6160,0 days 03:13:37.192000
std,35527.0424,0 days 09:15:54.013005570
min,3185.0000,0 days 00:02:17
25%,14250.0000,0 days 00:18:33
50%,31947.0000,0 days 00:37:05
75%,86400.0000,0 days 01:29:14
max,86400.0000,2 days 17:38:15


In [26]:
tasks.loc[tasks['analyst_id'] == 139][['file_duration', 'duration']].describe()

,file_duration,duration
count,108.000000,108
mean,39972.000000,0 days 03:47:01.351851851
std,34258.743462,0 days 11:33:03.756278505
min,3815.000000,0 days 00:04:09
25%,12639.750000,0 days 00:18:22.500000
50%,19369.000000,0 days 00:41:24.500000
75%,86400.000000,0 days 01:13:58.750000
max,86400.000000,2 days 19:13:45


It's difficult to determine who's more productive between analysts 139 and 149. If I put more weight on file_duration, it would be 139, but if I cared more about the number of completed tasks and average completion time, it would be 149.

### Conclusion

If we were to interpret productivity as the average file duration, it would be the analyst_id 39, whereas if we measure productivity by the quantitiy of completed tasks versus the time duration, it would be the analyst_id 139 and 149.

Although I personally think average file duration is more correlated to one's proficiency rather than productivity, it's still an important factor, so I'm going to choose __analyst_id 139 and 149__ as they seems to be in the sweet spot.